In [0]:
from pyspark.sql.functions import *

# Configuration for accessing CosmosDB
readConfig = {
  "spark.cosmos.accountEndpoint": "https://scc23nmp.documents.azure.com:443/",
  "spark.cosmos.accountKey": "3Stlau3NbZIGs97x5KOW8J9byciE0GxuiN3B3P8ykzivdccUzRhrp6xMUQ4SOHmQBGZWcJqK7WsPDalVZwXQcQ==",
  "spark.cosmos.database": "scc23dbnmp",
  "spark.cosmos.container": "auctions",
    # Getting only the id and owner from the auctions
  "spark.cosmos.read.customQuery": "SELECT a.id, a.owner FROM auctions a"
}

auctions = spark.read.format("cosmos.oltp").options(**readConfig) \
                    .option("spark.cosmos.read.inferSchema.enabled", "true") \
                    .load()
# Let's register the dataframe as a view
auctions.createOrReplaceTempView("auctions")

# Count the number of auctions for each owner
result = spark.sql("""SELECT owner, count(*) AS count FROM auctions
                          GROUP BY owner""")

# Write configuration
writeConfig = {
  "spark.cosmos.accountEndpoint": "https://scc23nmp.documents.azure.com:443/",
  "spark.cosmos.accountKey": "3Stlau3NbZIGs97x5KOW8J9byciE0GxuiN3B3P8ykzivdccUzRhrp6xMUQ4SOHmQBGZWcJqK7WsPDalVZwXQcQ==",
  "spark.cosmos.database": "scc23dbnmp",
  "spark.cosmos.container": "auctionsFreq",
}

# Write to Cosmos DB from the result DataFrame
result.write.format("cosmos.oltp").options(**writeConfig) \
              .mode("append") \
              .save()
